# 5. 变形
参考DataWhale：https://datawhalechina.github.io/joyful-pandas/build/html/%E7%9B%AE%E5%BD%95/ch5.html#melt

In [1]:
import numpy as np
import pandas as pd

## 1. 长宽表的变形
长宽表是相对来说的。对某一特征的长表（该特征为cell值），对某一特征的宽表（该特征的不同level为列名）。

In [2]:
#对性别的长表
pd.DataFrame({'Gender':['F','F','M','M'],
             'Height':[163,160,175,180]})

,Gender,Height
0,F,163
1,F,160
2,M,175
3,M,180


In [3]:
#对性别的宽表
pd.DataFrame({'Height: F':[163,160],
             'Height: M':[175,180]})

,Height: F,Height: M
0,163,175
1,160,180


### 1.1 pivot
长表变宽表。  
pivot的参数包括了index（行索引）、columns（列变列索引）、values（新表中的数值）。   
index和columns的每个combination对应的数值都必须是唯一的。

In [4]:
df=pd.DataFrame({'Class':[1,1,2,2], 'Name':['San Zhang','San Zhang','Si Li','Si Li'], 
                 'Subject':['Chinese','Math','Chinese','Math'],
                 'Grade':[80,75,90,85]})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [5]:
#以Name为行索引，将subject的每个level转换成列名，然后对应的数值是Grade。每位同学的每门学科的成绩表。
df.pivot(index='Name', columns='Subject', values='Grade')

Subject,Chinese,Math
Name,,
San Zhang,80,75
Si Li,90,85


In [6]:
#pivot的三个参数可以设置为列表---多级索引。
df = pd.DataFrame({'Class':[1,1,2,2,1,1,2,2], 
                   'Name':['San Zhang','San Zhang','Si Li','Si Li','San Zhang','San Zhang','Si Li','Si Li'], 
                   'Examination': ['Mid','Final','Mid','Final','Mid','Final','Mid','Final'], 
                   'Subject':['Chinese','Chinese','Chinese','Chinese','Math','Math','Math','Math'], 
                   'Grade':[80,75,85,65,90,85,92,88], 'Rank':[10,15,21,15,20,7,6,2]})
df

,Class,Name,Examination,Subject,Grade,Rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


In [7]:
#把Examination和Subject的combination转成列名
pivot_multi=df.pivot(index=['Class','Name'], columns=['Examination','Subject'], values=['Grade','Rank'])
pivot_multi

Grade                       Rank                   
Examination         Mid   Final  Mid Final     Mid   Final  Mid Final
Subject         Chinese Chinese Math  Math Chinese Chinese Math  Math
Class Name                                                           
1     San Zhang      80      75   90    85      10      15   20     7
2     Si Li          85      65   92    88      21      15    6     2

### 1.2 pivot_table
当value不满足唯一性条件时，pivot_table函数中的aggfunc参数定义使用的聚合函数。同时也可以计算汇总，使用margins=True（False default）。

In [8]:
df = pd.DataFrame({'Name':['San Zhang','San Zhang','San Zhang','San Zhang','Si Li','Si Li','Si Li','Si Li'],
                   'Subject':['Chinese','Chinese','Math','Math','Chinese','Chinese','Math','Math'],
                   'Grade':[80,90,100,90,70,80,85,95]})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


In [9]:
df.pivot_table(index='Name', columns='Subject', values='Grade', aggfunc='mean')

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


In [10]:
#aggfunc=function
df.pivot_table(index='Name', columns='Subject', values='Grade', aggfunc=lambda x: x.mean())

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


In [11]:
#margin
df.pivot_table(index='Name', columns='Subject', values='Grade', aggfunc='mean', margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,85,95.0,90.00
Si Li,75,90.0,82.50
All,80,92.5,86.25


In [12]:
#练一练
#不成立，因为margins计算应该是基于aggfunc之前的原表，那么平均之后再平均和整体平均是不一样的。

### 1.3 melt
宽表变长表。与pivot互逆。   
melt的参数包括id_vars（行索引）、value_vars（列索引变列），var_name（列索引变成的新列的列名）、value_name（剩余的数值的列名）。

In [13]:
df=pd.DataFrame({'Class':[1,2], 'Name':['San Zhang','Si Li'], 'Chinese':[80,90], 'Math':[80,75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [14]:
df_melted=df.melt(id_vars=['Class','Name'], value_vars=['Chinese','Math'], var_name='Subject', value_name='Grade')
df_melted

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


In [15]:
df_unmelted=df_melted.pivot(index=['Class','Name'], columns='Subject', values='Grade')
df_unmelted

,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,80
2,Si Li,90,75


### 1.4 wide_to_long
将列索引变成多列。   
wide_to_long的参数包括stubnames（新列名）、i（行索引）、j（变成索引行的列名）、sep（分隔符）、suffix（正则后缀）。

In [16]:
df = pd.DataFrame({'Class':[1,2], 'Name':['San Zhang','Si Li'], 'Chinese_Mid':[80,75], 'Math_Mid':[90,85],
                   'Chinese_Final':[80,75], 'Math_Final':[90,85]})
df

,Class,Name,Chinese_Mid,Math_Mid,Chinese_Final,Math_Final
0,1,San Zhang,80,90,80,90
1,2,Si Li,75,85,75,85


In [17]:
pd.wide_to_long(df, stubnames=['Chinese','Math'], i=['Class','Name'], j='Examination', sep='_', suffix='.+')

Chinese  Math
Class Name      Examination               
1     San Zhang Mid               80    90
                Final             80    90
2     Si Li     Mid               75    85
                Final             75    85

In [18]:
res=pivot_multi.copy()
res.columns=res.columns.map(lambda x: '_'.join(x))
res=res.reset_index()
res

,Class,Name,Grade_Mid_Chinese,Grade_Final_Chinese,Grade_Mid_Math,Grade_Final_Math,Rank_Mid_Chinese,Rank_Final_Chinese,Rank_Mid_Math,Rank_Final_Math
0,1,San Zhang,80,75,90,85,10,15,20,7
1,2,Si Li,85,65,92,88,21,15,6,2


In [19]:
res1=pd.wide_to_long(res, stubnames=['Grade','Rank'], i=['Class','Name'], j='Subject_Examination', sep='_', suffix='.+')
res1=res1.reset_index()
res1[['Subject','Examination']]=res1['Subject_Examination'].str.split('_', expand=True)
res1=res1[['Class','Name','Examination','Subject','Grade','Rank']].sort_values('Subject')
res1

,Class,Name,Examination,Subject,Grade,Rank
1,1,San Zhang,Chinese,Final,75,15
3,1,San Zhang,Math,Final,85,7
5,2,Si Li,Chinese,Final,65,15
7,2,Si Li,Math,Final,88,2
0,1,San Zhang,Chinese,Mid,80,10
2,1,San Zhang,Math,Mid,90,20
4,2,Si Li,Chinese,Mid,85,21
6,2,Si Li,Math,Mid,92,6


## 2. 索引的变形
### 2.1 stack与unstack
unstack：   
把行索引转成列索引；    
参数为移动的层号，默认转换最内层，也可以转换多层；    
转换的行索引和保留的行索引的combination必须是唯一的。

In [20]:
df=pd.DataFrame(np.ones((4,2)), 
                index=pd.Index([('A','cat','big'), ('A','dog','small'), ('B','cat','big'), ('B','dog','small')]),
                columns=['col_1','col_2'])
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [21]:
df.unstack()

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [22]:
#只转换行索引的第一层和第三层
df.unstack([0,2])

col_1                  col_2                 
        A          B           A          B      
      big small  big small   big small  big small
cat   1.0   NaN  1.0   NaN   1.0   NaN  1.0   NaN
dog   NaN   1.0  NaN   1.0   NaN   1.0  NaN   1.0

stack：   
把列索引转成行索引；
参数为移动的层号，默认转换最内层，也可以转换多层；    
转换的行索引和保留的行索引的combination必须是唯一的。

In [23]:
df=df.T
df

A          B      
       cat   dog  cat   dog
       big small  big small
col_1  1.0   1.0  1.0   1.0
col_2  1.0   1.0  1.0   1.0

In [24]:
df.stack()

A         B     
             cat  dog  cat  dog
col_1 big    1.0  NaN  1.0  NaN
      small  NaN  1.0  NaN  1.0
col_2 big    1.0  NaN  1.0  NaN
      small  NaN  1.0  NaN  1.0

### 2.2 聚合与变形的关系
普通的变形函数不会有值个数的改变，但是聚合会把多个值变为一个值，值的个数发生的变化。

## 3. 其他变形函数
### 3.1 crosstab
功能可以通过pivot_table实现，性能速度会较慢。默认会计算count值。

In [25]:
df=pd.read_csv('/Users/jie/Documents/Python/joyful-pandas-master/data/learn_pandas.csv')
df.head(3)

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22


In [26]:
#参数同df
pd.crosstab(index=df.School, columns=df.Transfer)

Transfer,N,Y
School,,
Fudan University,38,1
Peking University,28,2
Shanghai Jiao Tong University,53,0
Tsinghua University,62,4


In [27]:
#等价的pivot_table
df.pivot_table(index='School', columns='Transfer', values='Name', aggfunc='count')

Transfer,N,Y
School,,
Fudan University,38.0,1.0
Peking University,28.0,2.0
Shanghai Jiao Tong University,53.0,NaN
Tsinghua University,62.0,4.0


In [28]:
#练一练
#mean
%timeit -n 30 pd.crosstab(index=df.School, columns=df.Transfer, values=df.Height, aggfunc='mean')

6.18 ms ± 598 µs per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [29]:
%timeit -n 30 df.pivot_table(index='School', columns='Transfer', values='Height', aggfunc='mean')

5.3 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 30 loops each)


### 3.2 explode
对某一列的元素进行纵向的展开。展开的类型有list、tuple、Series、np.ndarray。

In [30]:
df_ex = pd.DataFrame({'A':[[1,2], 'my_str', {1,2}, pd.Series([3,4])], 'B':1})
df_ex

,A,B
0,"[1, 2]",1
1,my_str,1
2,"{1, 2}",1
3,0 3 1 4 dtype: int64,1


In [31]:
df_ex.explode('A')

,A,B
0,1,1
0,2,1
1,my_str,1
2,"{1, 2}",1
3,3,1
3,4,1


### 3.3 get_dummies
得到指示变量。

In [32]:
pd.get_dummies(df.Grade).head()

,Freshman,Junior,Senior,Sophomore
0,1,0,0,0
1,1,0,0,0
2,0,0,1,0
3,0,0,0,1
4,0,0,0,1


## 4. 练习
### Ex1: 美国非法药物数据集

In [33]:
df1=pd.read_csv('/Users/jie/Documents/Python/joyful-pandas-master/data/drugs.csv').sort_values([
    'State','COUNTY','SubstanceName'], ignore_index=True)
df1

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2011,KY,ADAIR,Buprenorphine,3
1,2012,KY,ADAIR,Buprenorphine,5
2,2013,KY,ADAIR,Buprenorphine,4
3,2014,KY,ADAIR,Buprenorphine,27
4,2015,KY,ADAIR,Buprenorphine,5
...,...,...,...,...,...
24057,2010,WV,WYOMING,Oxycodone,5
24058,2011,WV,WYOMING,Oxycodone,4
24059,2012,WV,WYOMING,Oxycodone,14
24060,2013,WV,WYOMING,Oxycodone,12


In [34]:
#1. 
df1_1=df1.copy()
df1_1_pivot=df1_1.pivot(index=['State','COUNTY','SubstanceName'], columns='YYYY', values='DrugReports')
df1_1_pivot=df1_1_pivot.reset_index()
df1_1_pivot

YYYY,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,NaN,3.0,5.0,4.0,27.0,5.0,7.0,10.0
1,KY,ADAIR,Codeine,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
2,KY,ADAIR,Fentanyl,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,KY,ADAIR,Heroin,NaN,NaN,1.0,2.0,NaN,1.0,NaN,2.0
4,KY,ADAIR,Hydrocodone,6.0,9.0,10.0,10.0,9.0,7.0,11.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
6209,WV,WOOD,Oxycodone,6.0,4.0,24.0,7.0,7.0,11.0,7.0,1.0
6210,WV,WOOD,Tramadol,NaN,NaN,NaN,NaN,1.0,NaN,4.0,3.0
6211,WV,WYOMING,Buprenorphine,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0
6212,WV,WYOMING,Hydrocodone,1.0,5.0,NaN,NaN,1.0,NaN,1.0,NaN


In [35]:
#2. 
df1_1_pivot.melt(id_vars=['State','COUNTY','SubstanceName'], value_vars=[2010,2011,2012,2013,2014,2015,2016,2017],
                var_name='YYYY', value_name='DrugReports').dropna(subset=['DrugReports'])

,State,COUNTY,SubstanceName,YYYY,DrugReports
4,KY,ADAIR,Hydrocodone,2010,6.0
6,KY,ADAIR,Methadone,2010,1.0
13,KY,ALLEN,Hydrocodone,2010,10.0
15,KY,ALLEN,Methadone,2010,4.0
17,KY,ALLEN,Oxycodone,2010,15.0
...,...,...,...,...,...
49702,WV,WOOD,Hydrocodone,2017,8.0
49704,WV,WOOD,Isobutyryl fentanyl,2017,3.0
49707,WV,WOOD,Oxycodone,2017,1.0
49708,WV,WOOD,Tramadol,2017,3.0


In [36]:
#3.
#pivot_table
df1_3=df1.copy()
df1_3.pivot_table(index='YYYY', columns='State', values='DrugReports', aggfunc='sum')

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376
2013,11148,26846,20409,11675,4046
2014,11081,30860,24904,9037,3280
2015,9865,37127,25651,8810,2571
2016,9093,42470,26164,10195,2548
2017,9394,46104,27894,10448,1614


In [37]:
pd.DataFrame(df1_3.groupby(['State','YYYY'])['DrugReports'].sum()).unstack(0)

DrugReports                           
State          KY     OH     PA     VA    WV
YYYY                                        
2010        10453  19707  19814   8685  2890
2011        10289  20330  19987   6749  3271
2012        10722  23145  19959   7831  3376
2013        11148  26846  20409  11675  4046
2014        11081  30860  24904   9037  3280
2015         9865  37127  25651   8810  2571
2016         9093  42470  26164  10195  2548
2017         9394  46104  27894  10448  1614

### Ex2：特殊的wide_to_long方法

In [38]:
df2=pd.DataFrame({'Class':[1,2], 'Name':['San Zhang','Si Li'], 'Chinese':[80,90], 'Math':[80,75]})
df2

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [39]:
df2.columns=['Class','Name','S-Chinese','S-Math']
df2

,Class,Name,S-Chinese,S-Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [40]:
df2_long=pd.wide_to_long(df2, stubnames='S', i=['Class','Name'], j='Subject', sep='-', suffix='.+').reset_index()
df2_long

,Class,Name,Subject,S
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,80
2,2,Si Li,Chinese,90
3,2,Si Li,Math,75


In [41]:
df2_long.columns=['Class','Name','Subject','Grade']
df2_long

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,80
2,2,Si Li,Chinese,90
3,2,Si Li,Math,75
